In [19]:
import sys
import json
import base64
import numpy as np
import os
from PIL import Image
import io
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
# from keras.models import load_model
# print(tf.__version__)
import random

In [4]:
!pip install -U -q segmentation-models
# !pip install -q numpy==1.18.5
# !pip install -q tensorflow==2.15.0
!pip install -q tensorflow==2.15.0

!pip install -q keras==2.15.0
# !pip install -q keras
!pip install -q tensorflow-estimator



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:

def crop_center(img, cropx, cropy):
    # Assuming img has shape (slices, height, width, channels)
    d, y, x, c = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    
    # Ensure the cropped image has the same number of channels
    return img[:, starty:starty+cropy, startx:startx+cropx, :]

def pad_slices(scan, target_slices= 30):
    # Get the current number of slices
    current_slices = scan.shape[0]
    # Check if padding is necessary
    if current_slices < target_slices:
        # Calculate padding amounts
        pad_before = (target_slices - current_slices) // 2
        pad_after = target_slices - current_slices - pad_before
        # Pad the scan with zeros on the slices axis (axis 0)
        padded_scan = np.pad(scan, pad_width=((pad_before, pad_after), (0, 0), (0, 0)), mode='constant', constant_values=0)
    elif current_slices > target_slices:
        # Calculate the cropping needed
        start = (current_slices - target_slices) // 2
        end = start + target_slices
        # Crop the scan to the target size
        padded_scan = scan[start:end, :, :]
    else:
        # If the number of slices is already equal to the target, no action is needed
        padded_scan = scan
    return padded_scan

def random_rotation(img, max_rotations):
    # Rotate image by a random number of 90-degree steps
    k = random.randint(0, max_rotations)  # Choose a random rotation
    return np.rot90(img, k=k, axes=(0, 1))  # Rotate on the (height, width) plane

def horizontal_flip(img, flip_prob):
    # Flip image horizontally with a given probability
    if random.random() < flip_prob:
        return np.flip(img, axis=1)  # Flip on the width axis
    return img



In [12]:
im = []
image_array = np.load('uploads/data_a.npy')
image_array_s = np.load('uploads/data_s.npy')
image_array_c = np.load('uploads/data_c.npy')

for i in range(len(image_array)):
    im.append(Image.fromarray(image_array[i].astype('uint8')))
for i in range(len(image_array_s)):
    im.append(Image.fromarray(image_array_s[i].astype('uint8')))
for i in range(len(image_array_c)):
    im.append(Image.fromarray(image_array_c[i].astype('uint8')))


In [16]:
im

[<PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 <PIL.Image.Image image mode=L siz

In [50]:
array_a = pad_slices(image_array)
array_s = pad_slices(image_array_s)
array_c = pad_slices(image_array_c)




In [51]:
all = np.stack((array_a, array_s, array_c), axis=-1)
all = np.array([crop_center(all, 160, 160) ])


In [56]:
max_rotations = 3  # This will allow for 0, 90, 180, and 270 degrees of rotation
flip_prob = 0.5

# rotated = random_rotation(all, max_rotations)
# flipped = horizontal_flip(rotated, flip_prob)
# all = flipped

batch = all

all.shape


(1, 30, 160, 160, 3)

In [ ]:
augmented_batch = np.empty_like(batch)
for scan_idx, scan in enumerate(batch):
    for slice_idx, img_slice in enumerate(scan):
        # No need to check for channel dimension for MRI data
        rotated = random_rotation(img_slice, max_rotations)
        flipped = horizontal_flip(rotated, flip_prob)
        augmented_batch[scan_idx, slice_idx] = flipped

# Update the original data array with the augmented batch
all = augmented_batch

In [31]:
for i in range(50):
    print(array_a[25][i])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  3  3  3
  4  4  5  5  6  6  5  4  5  7  8  7  6  6  8  8  8 10 10 10 10 11 10  9
 11 11  9  9 10 12 12 12 13 13 13 13 11  9 10  8  3  2  2  2  2  1  0  1
  1  1  0  0  1  1  1  1  1  1  1  1  1  1  0  1  1  1  1  1  1  1  1  1
  1  0  1  1  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1
 

In [31]:

# batch_min = all.min()
# batch_max = all.max()
# all = (all.astype(np.float32) - batch_min) / (batch_max - batch_min)


In [9]:
all.shape
for i in range(50):
    print(all[0][25][i])

[[ 43 225  43]
 [  4 176   4]
 [ 30  79  30]
 [ 72  54  72]
 [ 59  58  59]
 [ 68  55  68]
 [139  59 139]
 [148  64 148]
 [147  61 147]
 [145  62 145]
 [140  63 140]
 [132  62 132]
 [128  64 128]
 [ 94  71  94]
 [ 56  79  56]
 [ 36  99  36]
 [ 20 135  20]
 [ 29 149  29]
 [ 27 152  27]
 [ 26 167  26]
 [ 64 172  64]
 [ 93 174  93]
 [ 76 177  76]
 [ 71 175  71]
 [ 76 163  76]
 [106 152 106]
 [121 159 121]
 [120 152 120]
 [128 134 128]
 [ 95 116  95]
 [ 21  65  21]
 [  7  17   7]
 [ 49  16  49]
 [ 90  53  90]
 [ 80 104  80]
 [ 77 122  77]
 [ 86 112  86]
 [ 75 120  75]
 [ 85 120  85]
 [ 92 112  92]
 [ 86 116  86]
 [ 84 111  84]
 [ 77 109  77]
 [ 90 117  90]
 [ 85 115  85]
 [ 90 112  90]
 [104 114 104]
 [ 92 110  92]
 [ 88 111  88]
 [ 86 117  86]
 [ 77 117  77]
 [ 85 111  85]
 [ 95 110  95]
 [ 79 116  79]
 [ 79 121  79]
 [ 90 121  90]
 [ 82 119  82]
 [ 82 116  82]
 [ 83 113  83]
 [ 75 114  75]
 [ 82 114  82]
 [ 91 112  91]
 [ 84 113  84]
 [ 81 118  81]
 [ 83 115  83]
 [ 86 109  86]
 [ 77 115 

In [46]:
model_path = os.path.join('models', 'imageclassifier11.h5')
model = load_model(model_path)
prediction = model.predict(all)


ValueError: in user code:

    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/yvonnewang/opt/anaconda3/lib/python3.9/site-pa

In [41]:
prediction

array([[0.39975613, 0.31883916, 0.2785312 ]], dtype=float32)

In [35]:
prediction = prediction[0]
abnormal_pred = prediction[0]
acl_pred = prediction[1]
meniscus_pred = prediction[2]


if acl_pred > 0.55:
    print("ACL injury")
    print("type ", type(acl_pred))
elif abnormal_pred > 0.55:
    print("No ACL injury, but has abnormalities")

else:
    print("No ACL injury")

No ACL injury, but has abnormalities


In [17]:
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 30, 160, 160, 3)]    0         []                            
                                                                                                  
 conv3d_12 (Conv3D)          (None, 15, 80, 80, 32)       32960     ['input_3[0][0]']             
                                                                                                  
 batch_normalization_12 (Ba  (None, 15, 80, 80, 32)       128       ['conv3d_12[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_10 (Activation)  (None, 15, 80, 80, 32)       0         ['batch_normalization_12